In [1]:
import pandas as pd
import gurobipy
import os
from gurobipy import GRB, Model, quicksum
import itertools

In [2]:
os.chdir("C:\\Users\\julio\\OneDrive\\Escritorio\\G4 Capstone")

In [3]:
datos1 = pd.read_excel('Datos base G4.xlsx', sheet_name='Datos base G4')
nancagua = pd.read_excel('Datos base G4.xlsx', sheet_name='Bodega Nancagua')
machali = pd.read_excel('Datos base G4.xlsx', sheet_name='Bodega Machali')
chepica = pd.read_excel('Datos base G4.xlsx', sheet_name='Bodega Chepica')
transporte = pd.read_excel('Datos base G4.xlsx', sheet_name='Transporte')

In [4]:
costo_transporte = {}
for Cuartel, Machali, Chepica, Nancagua in zip(transporte['Cuartel'], transporte['Machali'], transporte['Chepica'], transporte['Nancagua']):
    costo_transporte[f'cuartel_{Cuartel}'] = {'Machali': Machali, 'Chepica': Chepica, 'Nancagua': Nancagua}

In [5]:
# Conjunto de cuarteles.
cuarteles = []
for i in range (1, 61):
    cuarteles.append(f'cuartel_{i}')

# Diccionario con el factor de conversion de toneladas a litros de cada cuartel.
conversion = {}

# Factor de penalización según día de cosecha.
penalizacion = {}

In [6]:
# Subconjuntos cepa-calidad. Cada subconjunto contiene los cuarteles que producen una cepa y calidad específica
G_variedad = []
G_reserva = []
G_premium = []
Ch_variedad = []
Ch_reserva = []
Ch_premium = []
SB_variedad = []
SB_reserva = []
SB_premium = []
C_variedad = []
C_reserva = []
C_premium = []
CS_variedad = []
CS_reserva = []
CS_premium = []
S_variedad = []
S_reserva = []
S_premium = []
M_variedad = []
M_reserva = []
M_premium = []
CF_variedad = []
CF_reserva = []
CF_premium = []
V_variedad = []
V_reserva = []
V_premium = []

In [7]:
# Parámetro binario que es 1 si se puede cosechar el cuartel i en el día t y 0 en caso contrario.
posible_cosechar = {}

# Total a cosechar en cada cuartel, con enfoque robusto da la misma cantidad para todos los cuarteles> 72,2 toneladas.
total_cosechar = {}

# Calidad de uva en el cuartel j.
productividad_cuartel = {}

# Precio de venta por cuartel
precio_venta = {}

In [8]:
for ID, Precio, Factor, Variedad, Dia_inicial, Dia_final in zip(datos1['ID'], datos1['Precio'], datos1['Factor'], datos1['Variedad'], datos1['Dia_inicial'], datos1['Dia_final']):
    total_cosechar[f'cuartel_{int(ID)}'] = 72.2
    precio_venta[f'cuartel_{ID}'] = Precio
    if Precio == 1000:
        productividad_cuartel[f'cuartel_{int(ID)}'] = 0.5
    elif Precio == 3000:
        productividad_cuartel[f'cuartel_{int(ID)}'] = 0.42
    elif Precio == 6000:
        productividad_cuartel[f'cuartel_{int(ID)}'] = 0.3

    conversion[f'cuartel_{int(ID)}'] = Factor
    
    penal = {}
    cosechar = {}
    for i in range(0, Dia_inicial + 1):
        if i+1 < Dia_inicial:
            penal[f'dia_{i+1}'] = 0
            cosechar[f'dia_{i+1}'] = 0

        elif i+1 == Dia_inicial:
            cosechar[f'dia_{i+1}'] = 1
            cosechar[f'dia_{i+2}'] = 1
            cosechar[f'dia_{i+3}'] = 1
            cosechar[f'dia_{i+4}'] = 1
            cosechar[f'dia_{i+5}'] = 1
            cosechar[f'dia_{i+6}'] = 1
            cosechar[f'dia_{i+7}'] = 1
            cosechar[f'dia_{i+8}'] = 1

            if Precio == 1000:
                penal[f'dia_{i+1}'] = 0.8
                penal[f'dia_{i+2}'] = 0.8875
                penal[f'dia_{i+3}'] = 0.95
                penal[f'dia_{i+4}'] = 0.9875
                penal[f'dia_{i+5}'] = 1
                penal[f'dia_{i+6}'] = 0.9833
                penal[f'dia_{i+7}'] = 0.9333
                penal[f'dia_{i+8}'] = 0.85
            elif Precio == 3000:
                penal[f'dia_{i+1}'] = 0.6
                penal[f'dia_{i+2}'] = 0.775
                penal[f'dia_{i+3}'] = 0.9
                penal[f'dia_{i+4}'] = 0.975
                penal[f'dia_{i+5}'] = 1
                penal[f'dia_{i+6}'] = 0.9667
                penal[f'dia_{i+7}'] = 0.8667
                penal[f'dia_{i+8}'] = 0.7
            else:
                penal[f'dia_{i+1}'] = 0.1
                penal[f'dia_{i+2}'] = 0.4938
                penal[f'dia_{i+3}'] = 0.775
                penal[f'dia_{i+4}'] = 0.9438
                penal[f'dia_{i+5}'] = 1
                penal[f'dia_{i+6}'] = 0.9111
                penal[f'dia_{i+7}'] = 0.6444
                penal[f'dia_{i+8}'] = 0.2
            
            for j in range(0, datos1['Dia_final'].max() - len(cosechar)):
                penal[f'dia_{i + j + 9}'] = 0
                cosechar[f'dia_{i + j + 9}'] = 0

        penalizacion[f'cuartel_{int(ID)}'] = penal
        posible_cosechar[f'cuartel_{int(ID)}'] = cosechar

    if Variedad == 'G':
        if Precio == 1000:
            G_variedad.append(f'cuartel_{int(ID)}')
        elif Precio == 3000:
            G_reserva.append(f'cuartel_{int(ID)}')
        else:
            G_premium.append(f'cuartel_{int(ID)}')
    elif Variedad == 'Ch':
        if Precio == 1000:
            Ch_variedad.append(f'cuartel_{int(ID)}')
        elif Precio == 3000:
            Ch_reserva.append(f'cuartel_{int(ID)}')
        else:
            Ch_premium.append(f'cuartel_{int(ID)}')
    elif Variedad == 'SB':
        if Precio == 1000:
            SB_variedad.append(f'cuartel_{int(ID)}')
        elif Precio == 3000:
            SB_reserva.append(f'cuartel_{int(ID)}')
        else:
            SB_premium.append(f'cuartel_{int(ID)}')
    elif Variedad == 'C':
        if Precio == 1000:
            C_variedad.append(f'cuartel_{int(ID)}')
        elif Precio == 3000:
            C_reserva.append(f'cuartel_{int(ID)}')
        else:
            C_premium.append(f'cuartel_{int(ID)}')
    elif Variedad == 'CS':
        if Precio == 1000:
            CS_variedad.append(f'cuartel_{int(ID)}')
        elif Precio == 3000:
            CS_reserva.append(f'cuartel_{int(ID)}')
        else:
            CS_premium.append(f'cuartel_{int(ID)}')
    elif Variedad == 'S':
        if Precio == 1000:
            S_variedad.append(f'cuartel_{int(ID)}')
        elif Precio == 3000:
            S_reserva.append(f'cuartel_{int(ID)}')
        else:
            S_premium.append(f'cuartel_{int(ID)}')
    elif Variedad == 'M':
        if Precio == 1000:
            M_variedad.append(f'cuartel_{int(ID)}')
        elif Precio == 3000:
            M_reserva.append(f'cuartel_{int(ID)}')
        else:
            M_premium.append(f'cuartel_{int(ID)}')
    elif Variedad == 'CF':
        if Precio == 1000:
            CF_variedad.append(f'cuartel_{int(ID)}')
        elif Precio == 3000:
            CF_reserva.append(f'cuartel_{int(ID)}')
        else:
            CF_premium.append(f'cuartel_{int(ID)}')
    elif Variedad == 'V':
        if Precio == 1000:
            V_variedad.append(f'cuartel_{int(ID)}')
        elif Precio == 3000:
            V_reserva.append(f'cuartel_{int(ID)}')
        else:
            V_premium.append(f'cuartel_{int(ID)}')

In [9]:
# Grupos de cuarteles.
grupos = ['grupo_1', 'grupo_2', 'grupo_3', 'grupo_4', 'grupo_5']
trabajadores_grupo = {'grupo_1': 100, 'grupo_2': 50, 'grupo_3': 100, 'grupo_4': 50, 'grupo_5': 100}
jornada = 8
grupo_cuarteles = {}
grupo1 = []
grupo2 = []
grupo3 = []
grupo4 = []
grupo5 = []
for i in range (0,60):
    if i+1 <= 16:
        grupo1.append(f'cuartel_{i+1}')
    elif i+1 <= 22:
        grupo2.append(f'cuartel_{i+1}')
    elif i+1 <= 32:
        grupo3.append(f'cuartel_{i+1}')
    elif i+1 <= 37:
        grupo4.append(f'cuartel_{i+1}')
    elif i+1 <= 60:
        grupo5.append(f'cuartel_{i+1}')

grupo_cuarteles['grupo_1'] = grupo1
grupo_cuarteles['grupo_2'] = grupo2
grupo_cuarteles['grupo_3'] = grupo3
grupo_cuarteles['grupo_4'] = grupo4
grupo_cuarteles['grupo_5'] = grupo5

In [10]:
cepa_calidad = {'G':{1000:G_variedad, 3000:G_reserva, 6000:G_premium},
                'Ch': {1000:Ch_variedad, 3000:Ch_reserva, 6000:Ch_premium},
                'SB': {1000:SB_variedad, 3000:SB_reserva, 6000:SB_premium},
                'C': {1000:C_variedad, 3000:C_reserva, 6000:C_premium},
                'CS': {1000:CS_variedad, 3000:CS_reserva, 6000:CS_premium},
                'S': {1000:S_variedad, 3000:S_reserva, 6000:S_premium},
                'M': {1000:M_variedad, 3000:M_reserva, 6000:M_premium},
                'CF': {1000:CF_variedad, 3000:CF_reserva, 6000:CF_premium},
                'V': {1000:V_variedad, 3000:V_reserva, 6000:V_premium}}

In [11]:
# Lista de todas las cepas que se producen en la viña
cepas = ["G", "Ch", "SB", "C", "CS", "S", "M", "CF", "V"]

# Lista de todas las bodegas donde se envían las uvas cosechadas
bodegas = ["Machali", "Chepica", "Nancagua"]

# Lista de todos los días del periodo de cosecha, hasta el mayor día final de cosecha.
periodo = []
for i in range(0, datos1['Dia_final'].max()):
    periodo.append(f'dia_{i+1}')

# Lista de todas las calidades de uva que se pueden cosechar
calidades = [1000, 3000, 6000]

# Costo por hora trabajada de cada trabajador.
costo_trabajador = 12000

# Big M
M = 1000000

# Multiplicador costo perdida
multiplicador_perdida = 10

# Llenado maximo y minimo de cada estanque
llenado_maximo = 0.95
llenado_minimo = 0.75

# Cantidad de estanques en cada bodega
nro_estanques = {'Machali': 14, 'Chepica': 9, 'Nancagua': 13}

In [12]:
#  Diccionario con los promedios de tiempos de fermentación de cada cepa.

promedio_fermentacion = {'C': 13, 'CF': 13, 'CS': 12, 'G': 13, 'M': 13, 'S': 13, 'V': 14, 'SB': 14, 'Ch': 14}

# Lista que agrupa los días en intervalos de X cantidad de días.

agrupacion = {}

for c in cepas:
    dias_agrupados = {}
    for i in range(0, datos1['Dia_final'].max() - promedio_fermentacion[c] + 1):
        d_a = []
        for j in range(0, promedio_fermentacion[c]):
            d_a.append(f'dia_{i+j+1}')
        dias_agrupados[f'agrupacion_{i+1}'] = d_a
    agrupacion[c] = dias_agrupados


In [13]:
agrupaciones_cepa = {}
for c in cepas:
    agrupaciones = []
    for i in range(0, len(agrupacion[c])):
        agrupaciones.append(f'agrupacion_{i+1}')
    agrupaciones_cepa[c] = agrupaciones

In [14]:
lista_estanques = []
lista_estanques_machali = []
lista_estanques_nancagua = []
lista_estanques_chepica = []
dic_estanques = {}
estanques = {}
estanques_nancagua = {}
estanques_machali = {}
estanques_chepica = {}
for ID, capacidad in zip(machali['Id'], machali['capacidad tanques (miles de litros)']):
    estanques_machali[f'estanque_machali_{ID}'] = capacidad
    lista_estanques.append(f'estanque_machali_{ID}')
    lista_estanques_machali.append(f'estanque_machali_{ID}')
dic_estanques['Machali'] = lista_estanques_machali
estanques['Machali'] = estanques_machali

for ID, capacidad in zip(nancagua['Id'], nancagua['capacidad tanques (miles de litros)']):
    estanques_nancagua[f'estanque_nancagua_{ID}'] = capacidad
    lista_estanques.append(f'estanque_nancagua_{ID}')
    lista_estanques_nancagua.append(f'estanque_nancagua_{ID}')
dic_estanques['Nancagua'] = lista_estanques_nancagua
estanques['Nancagua'] = estanques_nancagua

for ID, capacidad in zip(chepica['Id'], chepica['capacidad tanques (miles de litros)']):
    estanques_chepica[f'estanque_chepica_{ID}'] = capacidad
    lista_estanques.append(f'estanque_chepica_{ID}')
    lista_estanques_chepica.append(f'estanque_chepica_{ID}')
dic_estanques['Chepica'] = lista_estanques_chepica
estanques['Chepica'] = estanques_chepica

In [15]:
capacidad_maxima = {'Nancagua': nancagua['capacidad tanques (miles de litros)'].sum()*llenado_maximo, 'Chepica': chepica['capacidad tanques (miles de litros)'].sum()*llenado_maximo, 'Machali': machali['capacidad tanques (miles de litros)'].sum()*llenado_maximo}
capacidad_minima = {'Nancagua': min(nancagua['capacidad tanques (miles de litros)'])*llenado_minimo, 'Chepica': min(chepica['capacidad tanques (miles de litros)'])*llenado_minimo, 'Machali': min(machali['capacidad tanques (miles de litros)'])*llenado_minimo}

In [16]:
# Se setea una demanda para poder trabajar con el modelo cuando se itere con la simulacion

demanda = {'G': {1000: len(cepa_calidad['G'][1000])*72.2/20, 3000: len(cepa_calidad['G'][3000])*72.2/20, 6000: len(cepa_calidad['G'][6000])*72.2/20},
           'Ch': {1000: len(cepa_calidad['Ch'][1000])*72.2/20, 3000: len(cepa_calidad['Ch'][3000])*72.2/20, 6000: len(cepa_calidad['Ch'][6000])*72.2/20},
           'SB': {1000: len(cepa_calidad['SB'][1000])*72.2/20, 3000: len(cepa_calidad['SB'][3000])*72.2/20, 6000: len(cepa_calidad['SB'][6000])*72.2/20},
           'C': {1000: len(cepa_calidad['C'][1000])*72.2/20, 3000: len(cepa_calidad['C'][3000])*72.2/20, 6000: len(cepa_calidad['C'][6000])*72.2/20},
           'CS': {1000: len(cepa_calidad['CS'][1000])*72.2/20, 3000: len(cepa_calidad['CS'][3000])*72.2/20, 6000: len(cepa_calidad['CS'][6000])*72.2/20},
           'S': {1000: len(cepa_calidad['S'][1000])*72.2/20, 3000: len(cepa_calidad['S'][3000])*72.2/20, 6000: len(cepa_calidad['S'][6000])*72.2/20},
           'M': {1000: len(cepa_calidad['M'][1000])*72.2/20, 3000: len(cepa_calidad['M'][3000])*72.2/20, 6000: len(cepa_calidad['M'][6000])*72.2/20},
           'CF': {1000: len(cepa_calidad['CF'][1000])*72.2/20, 3000: len(cepa_calidad['CF'][3000])*72.2/20, 6000: len(cepa_calidad['CF'][6000])*72.2/20},
           'V': {1000: len(cepa_calidad['V'][1000])*72.2/20, 3000: len(cepa_calidad['V'][3000])*72.2/20, 6000: len(cepa_calidad['V'][6000])*72.2/20}}

In [17]:
model = Model("Cosecha Vino")
model.setParam('MIPGap', 0.0005)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-02
Set parameter MIPGap to value 0.0005


In [18]:
# VARIABLES

# Variable que indica si la bodega b recibe uvas de la cepa c y calidad k en el día t.
R_btck = model.addVars(bodegas, periodo, cepas, calidades, vtype=GRB.BINARY, name="R_btck")

# Variable que indica la cantidad de uvas cosechadas en el periodo t, que son llevadas desde el cuartel j a la bodega b.
X_bjt = model.addVars(bodegas, cuarteles, periodo, vtype=GRB.CONTINUOUS, name="X_bjt")

# Variable de utilizacion de tanques en bodegas
Y_btcke = model.addVars(bodegas, periodo, cepas, calidades, lista_estanques, vtype=GRB.BINARY, name="Y_btcke")

# Variable que indica la horas/hombre asignadas al cuartel j en el periodo t.
N_jt = model.addVars(cuarteles, periodo, vtype=GRB.CONTINUOUS, name="N_jt")

In [19]:
#Restriccion: Un estanque no puede usarse dos veces en al menos X cantidad de dias

model.addConstrs(quicksum(Y_btcke[b,t,c,k,e] for c in cepas for k in calidades for b in bodegas) <= 1 for bo in bodegas for t in periodo for e in dic_estanques[bo]);

model.addConstrs(quicksum(Y_btcke[b,t,c,k,e] for c in cepas for t in agrupacion[co][a] for k in calidades) <= 1 for b in bodegas for co in cepas for e in dic_estanques[b] for a in agrupaciones_cepa[co]);

In [20]:
# Restriccion: Lo que se envia a las bodegas debe poder almacenarse en su totalidad en alguna combinación de estanques.

model.addConstrs(quicksum(X_bjt[b,j,t]*conversion[j] for j in cepa_calidad[c][k]) >= quicksum(Y_btcke[b,t,c,k,e]*estanques[b][e] for e in dic_estanques[b])*llenado_minimo for b in bodegas for t in periodo for c in cepas for k in calidades);
model.addConstrs(quicksum(X_bjt[b,j,t]*conversion[j] for j in cepa_calidad[c][k]) <= quicksum(Y_btcke[b,t,c,k,e]*estanques[b][e] for e in dic_estanques[b])*llenado_maximo for b in bodegas for t in periodo for c in cepas for k in calidades);

In [21]:
#Restriccion: No se puede cosechar mas que el total disponible en cada cuartel.

model.addConstrs(quicksum(X_bjt[b,j,t] for b in bodegas for t in periodo) <= total_cosechar[j] for j in cuarteles);

In [22]:
# Restriccion: La cantidad cosechada debe abastecer la demanda

model.addConstrs(quicksum(X_bjt[b,j,t] for b in bodegas for t in periodo for j in cepa_calidad[c][k]) >= demanda[c][k] for c in cepas for k in calidades);

In [23]:
# Restriccion: La cantidad cosechada debe ser mayor que el mínimo de cada bodega

model.addConstrs(quicksum(X_bjt[b,j,t]*conversion[j] for j in cepa_calidad[c][k]) >= capacidad_minima[b]*R_btck[b,t,c,k] for b in bodegas for t in periodo for c in cepas for k in calidades);

In [24]:
# Restriccion: auxiliar para ajustar el valor de la variable binaria R_btck

model.addConstrs(R_btck[b,t,c,k]*M >= quicksum(X_bjt[b,j,t] for j in cepa_calidad[c][k]) for b in bodegas for t in periodo for c in cepas for k in calidades);

In [25]:
# Restriccion: Solo se cosecha el cuartel si es apto segun la ventana de cosecha.

model.addConstrs(quicksum(X_bjt[b,j,t] for b in bodegas) <= M*posible_cosechar[j][t] for j in cuarteles for t in periodo);

In [26]:
# Restriccion: La cantidad de horas-hombre asignadas por grupo de cuarteles no puede superar la cantidad maxima disponible.

model.addConstrs(quicksum(N_jt[j,t] for j in grupo_cuarteles[g]) <= trabajadores_grupo[g]*jornada for g in grupos for t in periodo);

In [27]:
# Restriccion: La cosecha debe ser igual a la productividad de los trabajadores 

model.addConstrs(quicksum(X_bjt[b,j,t] for b in bodegas) == N_jt[j,t]*productividad_cuartel[j] for j in cuarteles for t in periodo);

In [28]:
# Funcion objetivo

obj = quicksum(X_bjt[b,j,t]*conversion[j]*1000*precio_venta[j]*penalizacion[j][t] for b in bodegas for j in cuarteles for t in periodo) - quicksum((total_cosechar[j] - quicksum(X_bjt[b,j,t] for b in bodegas for t in periodo))*precio_venta[j]*conversion[j]*1000*multiplicador_perdida for j in cuarteles) - quicksum(N_jt[j,t]*costo_trabajador for j in cuarteles for t in periodo) - quicksum(X_bjt[b,j,t]*costo_transporte[j][b]*1000 for b in bodegas for t in periodo for j in cuarteles)

In [29]:
model.setObjective(obj, GRB.MAXIMIZE)

In [30]:
model.optimize()

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (win64)

CPU model: AMD Ryzen 7 5800H with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 60038 rows, 255723 columns and 8240574 nonzeros
Model fingerprint: 0x125211ef
Variable types: 18960 continuous, 236763 integer (236763 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+06]
  Objective range  [1e+04, 6e+07]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective -8.57119e+10
Presolve removed 54076 rows and 244310 columns
Presolve time: 3.67s
Presolved: 5962 rows, 11413 columns, 139829 nonzeros
Variable types: 1446 continuous, 9967 integer (9967 binary)
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 1.040515e+10, 236 iterations, 0.08 seconds (0.08 work

In [31]:
model.printAttr('X')


    Variable            X 
-------------------------
R_btck[Machali,dia_16,G,1000]            1 
R_btck[Machali,dia_25,V,3000]            1 
R_btck[Machali,dia_27,CS,1000]            1 
R_btck[Machali,dia_29,Ch,6000]            1 
R_btck[Machali,dia_29,CS,3000]            1 
R_btck[Machali,dia_29,CS,6000]            1 
R_btck[Machali,dia_32,SB,3000]            1 
R_btck[Machali,dia_32,CS,3000]            1 
R_btck[Machali,dia_34,SB,3000]            1 
R_btck[Machali,dia_34,CS,6000]            1 
R_btck[Machali,dia_35,CS,3000]            1 
R_btck[Machali,dia_37,CS,3000]            1 
R_btck[Machali,dia_39,SB,6000]            1 
R_btck[Machali,dia_40,Ch,1000]            1 
R_btck[Machali,dia_44,SB,3000]            1 
R_btck[Machali,dia_46,M,6000]            1 
R_btck[Machali,dia_47,Ch,3000]            1 
R_btck[Machali,dia_49,CS,6000]            1 
R_btck[Machali,dia_50,C,1000]            1 
R_btck[Machali,dia_54,C,3000]            1 
R_btck[Machali,dia_54,CS,3000]            1 
R_btck

In [39]:
var_names_X = []
var_bodega_X = []
var_cuartel_X = []
var_dia_X = []
var_values_X = []
dia_min_X = []

var_names_N = []
var_cuartel_N = []
var_dia_N = []
var_values_N = []

for var in model.getVars():
    if var.X > 0 and var.varName[0] == 'X':
        splitted_var_X = var.VarName.split(',')
        splitted_var2_X = splitted_var_X[0].split('[')
        var_names_X.append(splitted_var2_X[0])
        var_bodega_X.append(splitted_var2_X[1])
        var_cuartel_X.append(splitted_var_X[1])
        var_dia_X.append(splitted_var_X[2].strip(']'))
        var_values_X.append(var.x)

    elif var.X > 0 and var.varName[0] == 'N':
        splitted_var_N = var.VarName.split(',')
        splitted_var2_N = splitted_var_N[0].split('[')
        var_names_N.append(splitted_var2_N[0])
        var_cuartel_N.append(splitted_var2_N[1])
        var_dia_N.append(splitted_var_N[1])
        var_values_N.append(var.x*costo_trabajador)

var_values2_X = []
for cuartel, valor in zip(var_cuartel_X, var_values_X):
    var_values2_X.append((valor)*conversion[cuartel])

In [38]:
df_cosecha = pd.DataFrame({'Variable': var_names_X, 'Cuartel': var_cuartel_X, 'Dia': var_dia_X, 'Bodega': var_bodega_X, 'Valor': var_values2_X})

df_trabajadores = pd.DataFrame({'Variable': var_names_N, 'Cuartel': var_cuartel_N, 'Dia': var_dia_N, 'Valor': var_values_N})

In [40]:
open("solucion_base.csv","w")
df_cosecha.to_csv("solucion_base.csv", index=False, mode="a")

open("trabajadores_base.csv","w")
df_trabajadores.to_csv("trabajadores_base.csv", index=False, mode="a")

In [35]:
df_cosecha['Valor'].sum()

3754.4000000000005